In [45]:
# %load_ext autoreload
# %autoreload 2
from openmsimodel.db.open_db import OpenDB
from openmsimodel.graph.open_graph import OpenGraph
from openmsimodel.workflow.birdshot_workflow import BIRDSHOTWorfklow
import networkx as nx
import json
from pathlib import Path

# Workflow

In [46]:
curr_pwd = Path().parent.absolute()
root= curr_pwd / "input/data/AAB" # put data and Sample Data folders inside input
output = curr_pwd / "output/AAB"
iteration = "AAB"
aab_synthesis_path = curr_pwd / "input/Sample Data/Iteration2_AAB/HTMDEC AAB Summary Synthesis Results.xlsx"
aab_srjt_path = curr_pwd / "input/Sample Data/Iteration2_AAB/Copy of HTMDEC AAB-VAM NI SRJT Set Summary.xlsx"
aab_birdshot_workflow = BIRDSHOTWorfklow(root, output=output,iteration=iteration,synthesis_path=aab_synthesis_path, srjt_path=aab_srjt_path)
aab_birdshot_workflow.build()

convert /srv/hemi01-j01/openmsimodel/examples/birdshot/input/Sample Data/Iteration2_AAB/Copy of HTMDEC AAB-VAM NI SRJT Set Summary.xlsx -> /srv/hemi01-j01/openmsimodel/examples/birdshot/output/AAB/Copy of HTMDEC AAB-VAM NI SRJT Set Summary.csv using filter : Text - txt - csv (StarCalc)
no ingredients and process for block 'Infer Compositions' were linked. 
no material and process for block 'Infer Compositions' were linked. 


Value {'units': 'megapascal', 'nominal': -1666.4729082726535, 'type': 'nominal_real'} is inconsistent with template Tensile
Value {'units': 'megapascal', 'nominal': -1666.4729082726535, 'type': 'nominal_real'} is inconsistent with template Tensile
Value {'units': 'megapascal', 'nominal': -1948.45757317272, 'type': 'nominal_real'} is inconsistent with template Tensile
Value {'units': 'megapascal', 'nominal': -1948.45757317272, 'type': 'nominal_real'} is inconsistent with template Tensile
Value {'category': 'BCC', 'type': 'nominal_categorical'} is inconsistent with template XRD
Value {'category': 'BCC', 'type': 'nominal_categorical'} is inconsistent with template XRD
Value {'category': 'BCC', 'type': 'nominal_categorical'} is inconsistent with template XRD
Value {'category': 'BCC', 'type': 'nominal_categorical'} is inconsistent with template XRD
Value {'units': 'megapascal', 'nominal': -22333.502289733286, 'type': 'nominal_real'} is inconsistent with template Tensile
Value {'units': 'meg

no material and process for block 'Infer Compositions' were linked. 


### Dive into

In [47]:
aab_birdshot_workflow.terminal_material

{'notes': None, 'spec': {'notes': None, 'file_links': [], 'tags': [], 'name': 'Infer comp. using B.O.', 'uids': {'auto': '06b26efa-b138-42c7-9a81-90000b34c5d8'}, 'conditions': [], 'parameters': [], 'template': {'description': 'Applying Bayezian Optimization based on multi-inputs/multi-output\n        objectives to generated new composition space, with data extracted \n        from summary sheet aggregated by various experimental teams\n        ', 'tags': [], 'allowed_names': [], 'name': 'Infer Compositions', 'uids': {'auto': '7ac2650e-7d8d-453a-90a5-6a8e5004c283', 'persistent_id': 63}, 'allowed_labels': [], 'conditions': [], 'parameters': [], 'type': 'process_template'}, 'type': 'process_spec', 'ingredients': ["<IngredientSpec 'Summary sheet Ing.'>"], 'output_material': None}, 'file_links': [], 'tags': [], 'name': 'Infer comp. using B.O.', 'uids': {'auto': '0c14eef2-20b0-429c-9550-b5e3c7d97bc6'}, 'conditions': [], 'parameters': [], 'source': None, 'type': 'process_run', 'ingredients': 

In [48]:
print(list(aab_birdshot_workflow.subs.keys()))
print("nb of blocks: {}".format(len(aab_birdshot_workflow.subs.keys())))

['Infer Compositions', 'Selecting Composition AAB01', 'Aggregating Co for AAB01-VAM-A', 'Aggregating Cr for AAB01-VAM-A', 'Aggregating Fe for AAB01-VAM-A', 'Aggregating Ni for AAB01-VAM-A', 'Aggregating V for AAB01-VAM-A', 'Aggregating Al for AAB01-VAM-A', 'Mix elements of AAB01', 'Arc Melting of Alloy Alloy (AAB01-VAM-A)', 'Homogenization of Alloy Alloy (AAB01-VAM-A)', 'Forging of Alloy Alloy (AAB01-VAM-A)', 'Setting up of Traveler Alloy (AAB01-VAM-A)', 'Set Alloy (AAB01-VAM-A) T. Sample for SEM (T02) charact.', 'Set Alloy (AAB01-VAM-A) T. Sample for XRD (T03) charact.', 'Set Alloy (AAB01-VAM-A) T. Sample for NI (T04) charact.', 'Set Alloy (AAB01-VAM-A) T. Sample for Tensile (T08) charact.', 'Set Alloy (AAB01-VAM-A) T. Sample for Tensile (T09) charact.', 'Selecting Composition AAB02', 'Aggregating Co for AAB02-VAM-A', 'Aggregating Cr for AAB02-VAM-A', 'Aggregating Fe for AAB02-VAM-A', 'Aggregating Ni for AAB02-VAM-A', 'Aggregating V for AAB02-VAM-A', 'Aggregating Al for AAB02-VAM-A', 

In [49]:
procedural_blocks = [obj.name for obj in aab_birdshot_workflow.subs.values() if obj.type=='procedural']
print(procedural_blocks)
print(len(procedural_blocks))

['Infer Compositions', 'Aggregating Co for AAB01-VAM-A', 'Aggregating Cr for AAB01-VAM-A', 'Aggregating Fe for AAB01-VAM-A', 'Aggregating Ni for AAB01-VAM-A', 'Aggregating V for AAB01-VAM-A', 'Aggregating Al for AAB01-VAM-A', 'Aggregating Co for AAB02-VAM-A', 'Aggregating Cr for AAB02-VAM-A', 'Aggregating Fe for AAB02-VAM-A', 'Aggregating Ni for AAB02-VAM-A', 'Aggregating V for AAB02-VAM-A', 'Aggregating Al for AAB02-VAM-A', 'Aggregating Co for AAB03-VAM-A', 'Aggregating Cr for AAB03-VAM-A', 'Aggregating Fe for AAB03-VAM-A', 'Aggregating Ni for AAB03-VAM-A', 'Aggregating V for AAB03-VAM-A', 'Aggregating Al for AAB03-VAM-A', 'Aggregating Co for AAB04-VAM-A', 'Aggregating Cr for AAB04-VAM-A', 'Aggregating Fe for AAB04-VAM-A', 'Aggregating Ni for AAB04-VAM-A', 'Aggregating V for AAB04-VAM-A', 'Aggregating Al for AAB04-VAM-A', 'Aggregating Co for AAB05-VAM-A', 'Aggregating Cr for AAB05-VAM-A', 'Aggregating Fe for AAB05-VAM-A', 'Aggregating Ni for AAB05-VAM-A', 'Aggregating V for AAB05-VAM-

In [50]:
arc_melting_aab04_vam_a_all = aab_birdshot_workflow.subs['Arc Melting of Alloy Alloy (AAB04-VAM-A)'].all()
print(arc_melting_aab04_vam_a_all)
arc_melting_aab04_vam_a_all_gemd = aab_birdshot_workflow.subs['Arc Melting of Alloy Alloy (AAB04-VAM-A)'].all_gemd()
# print()
print(len(arc_melting_aab04_vam_a_all_gemd))

[<openmsimodel.entity.base.ingredient.Ingredient object at 0x7f3a51fcc490>, <openmsimodel.entity.processes.birdshot.arc_melting.ArcMelting object at 0x7f3a51fcc7c0>, <openmsimodel.entity.materials.birdshot.alloy.Alloy object at 0x7f3a51fc3550>, <openmsimodel.entity.measurements.birdshot.weighting.Weighting object at 0x7f3a51fd06d0>]
11


multiple dumping approaches

In [33]:
aab_birdshot_workflow.thin_dumps(aab_birdshot_workflow.terminal_material, overwrite=True) # dumps into a single folder using a terminal GEMD object and GEMD dump methods 

Executing thin dumps...
Time elapsed: 9.32119369506836


In [9]:
aab_birdshot_workflow.thin_structured_dumps() # dumps into a nice COMPOSITION_ID/FABRICATION_METHOD/BATCH structure, with in-house methods

Executing thin structured dumps...
Time elapsed: 21.8549063205719


# Viz

visualize the whole workflow...

In [34]:
dirpath = curr_pwd / "output/AAB/unstructured"
output = curr_pwd / "output/AAB"
assets_to_add = {
            "add_attributes": 1,
            "add_file_links": 1,
            "add_tags": 1,
        }
open_graph = OpenGraph("unstructured",dirpath=dirpath,output=output)
all_G, all_relabeled_G, all_name_mapping = open_graph.build_graph(
    assets_to_add=assets_to_add,
    add_separate_node=0,
    which='run',
)
open_graph.launch(all_relabeled_G)

-- Building runs of /srv/hemi01-j01/openmsimodel/examples/birdshot/output/AAB/unstructured
Extracting folder...
0 gemd objects processed...
1000 gemd objects processed...
2000 gemd objects processed...
-- Saved graph to /srv/hemi01-j01/openmsimodel/examples/birdshot/output/AAB/unstructured_run_graph.dot and /srv/hemi01-j01/openmsimodel/examples/birdshot/output/AAB/unstructured_run_graph.svg
-- Analysis --
cycles in the graph: []
nb of disregarded elements: 35/2392
number of connected components: 1166
nb of isolates: 0


GraphWidget(layout=Layout(height='500px', width='100%'))

... or an important workflow branch...

In [11]:
dirpath = curr_pwd / "output/AAB/structured/AAB01/VAM/A/thin_jsons"
open_graph = OpenGraph("aab01_vam_a_thin",dirpath=dirpath,output=output)
G, relabeled_G, name_mapping = open_graph.build_graph(
    assets_to_add=assets_to_add,
    add_separate_node=0,
    which='run',
)
open_graph.launch(relabeled_G)

-- Building runs of /srv/hemi01-j01/openmsimodel/examples/birdshot/output/AAB/structured/AAB01/VAM/A/thin_jsons
Extracting folder...
0 gemd objects processed...
-- Saved graph to /srv/hemi01-j01/openmsimodel/examples/birdshot/output/AAB/aab01_vam_a_thin_run_graph.dot and /srv/hemi01-j01/openmsimodel/examples/birdshot/output/AAB/aab01_vam_a_thin_run_graph.svg
-- Analysis --
cycles in the graph: []
nb of disregarded elements: 35/195
number of connected components: 70
nb of isolates: 0


GraphWidget(layout=Layout(height='500px', width='100%'))

... or a subworkflow...

In [12]:
# print(arc_melting_aab04_vam_a_all_gemd)
open_graph = OpenGraph("arcmelting_block_aab04_vam_a", dirpath=arc_melting_aab04_vam_a_all[1].all_gemd(),output=output)
G, relabeled_G, name_mapping = open_graph.build_graph(
    assets_to_add=assets_to_add,
    add_separate_node=0,
    which='run',
)
open_graph.launch(relabeled_G)

-- Building runs of list with 3 items
Extracting list...
0 gemd objects processed...
-- Saved graph to /srv/hemi01-j01/openmsimodel/examples/birdshot/output/AAB/arcmelting_block_aab04_vam_a_run_graph.dot and /srv/hemi01-j01/openmsimodel/examples/birdshot/output/AAB/arcmelting_block_aab04_vam_a_run_graph.svg
-- Analysis --
cycles in the graph: []
nb of disregarded elements: 0/3
number of connected components: 1
nb of isolates: 1


GraphWidget(layout=Layout(height='500px', width='100%'))

... or a a BaseElement...

In [13]:
open_graph = OpenGraph("arcmelting_block_aab04_vam_a", dirpath=arc_melting_aab04_vam_a_all_gemd,output=output)
G, relabeled_G, name_mapping = open_graph.build_graph(
    assets_to_add=assets_to_add,
    add_separate_node=0,
    which='run',
)
open_graph.launch(relabeled_G)

-- Building runs of list with 11 items
Extracting list...
0 gemd objects processed...
-- Saved graph to /srv/hemi01-j01/openmsimodel/examples/birdshot/output/AAB/arcmelting_block_aab04_vam_a_run_graph.dot and /srv/hemi01-j01/openmsimodel/examples/birdshot/output/AAB/arcmelting_block_aab04_vam_a_run_graph.svg
-- Analysis --
cycles in the graph: []
nb of disregarded elements: 0/11
number of connected components: 5
nb of isolates: 0


GraphWidget(layout=Layout(height='500px', width='100%'))

... or a single elements with its descendents and predecessors

In [14]:
import random
identifier = random.choice(list(all_name_mapping.keys()))
print(identifier)
print(all_name_mapping[identifier])
functions = [nx.descendants, nx.ancestors]
try: # doesn't always run, depending on the randomly selected element
    identifier_G = OpenGraph.extract_subgraph(all_G, identifier, func=functions)
    identifier_G = OpenGraph.map_to_graphviz(identifier_G, all_name_mapping)
    open_graph.launch(identifier_G)
except Exception as e:
    print(e)

a8c6ed44-2b0d-403d-907d-e457abcd6c06
Alloy (AAB01-VAM-A): Traveler Ing.,  a8c
The node a8c6ed44-2b0d-403d-907d-e457abcd6c06 is not in the digraph.


TODO: SVG Viewer

# DB

In [41]:
db = OpenDB(database_name='GEMD', private_path='/home/arachid1/.private/gemd_db.json',output=curr_pwd / "output/queries_log")
model_id =11

Test query retrieving all measurement runs

In [42]:
query = f""" SELECT * FROM GEMDObject WHERE model_id={model_id} AND gemd_type='measurement_run' """
_all_measurements = db.gemd_db.execute_query(query)
db.print_and_dump(_all_measurements, query, "_all_measurements", dump=False)

                                      uid  model_id        gemd_type   
0    228a386d-2e90-4b8b-ada6-4950a16cee9a        11  measurement_run  \
1    2316a913-20a1-45e0-b905-275857e2c98d        11  measurement_run   
2    2393037a-d4e4-4f90-a775-d26c8975169f        11  measurement_run   
3    76384e60-e86e-413a-9693-7e36769ad0a6        11  measurement_run   
4    76a30c41-0d7a-4b18-8ed8-04ef1ca3a15e        11  measurement_run   
..                                    ...       ...              ...   
196  fe25bc50-7563-468c-bbfc-3ef14442600d        11  measurement_run   
197  33d2bd21-4e0f-46d9-a390-28bfcbd41d47        11  measurement_run   
198  34f81cb9-a738-4ffc-823c-1dcae0c2148d        11  measurement_run   
199  f4f8ecfd-b578-45e9-be24-c4a13c94a44f        11  measurement_run   
200  f7f9eb96-f4d7-44e4-9bc0-44358af46d8a        11  measurement_run   

                                               context  
0    {"conditions": [], "file_links": [], "material...  
1    {"conditions": [

,uid,model_id,gemd_type,context
0,228a386d-2e90-4b8b-ada6-4950a16cee9a,11,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
1,2316a913-20a1-45e0-b905-275857e2c98d,11,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
2,2393037a-d4e4-4f90-a775-d26c8975169f,11,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
3,76384e60-e86e-413a-9693-7e36769ad0a6,11,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
4,76a30c41-0d7a-4b18-8ed8-04ef1ca3a15e,11,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
...,...,...,...,...
196,fe25bc50-7563-468c-bbfc-3ef14442600d,11,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
197,33d2bd21-4e0f-46d9-a390-28bfcbd41d47,11,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
198,34f81cb9-a738-4ffc-823c-1dcae0c2148d,11,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
199,f4f8ecfd-b578-45e9-be24-c4a13c94a44f,11,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."


In [37]:
c = _all_measurements.iloc[1]['context']
print(json.dumps(json.loads(c), indent=3))

{
   "conditions": [],
   "file_links": [],
   "material": {
      "id": "ea7259bc-70e3-45dd-8556-26720a3f8fdb",
      "scope": "auto",
      "type": "link_by_uid"
   },
   "name": "SRJT charact. for AAB02",
   "notes": null,
   "parameters": [],
   "properties": [
      {
         "file_links": [],
         "name": "Strain Rate Sensitivity Exponent",
         "notes": "",
         "origin": "measured",
         "template": null,
         "type": "property",
         "value": {
            "nominal": 0.0120481190777778,
            "type": "nominal_real",
            "units": "dimensionless"
         }
      },
      {
         "file_links": [],
         "name": "Strain Rate Sensitivity Exponent Std. Dev.",
         "notes": "",
         "origin": "measured",
         "template": null,
         "type": "property",
         "value": {
            "nominal": 0.0020231034576555,
            "type": "nominal_real",
            "units": "dimensionless"
         }
      },
      {
         "

a couple of invalid jsons that need fixing

In [38]:
query = f""" SELECT *
FROM GEMDObject 
WHERE model_id={model_id} and isjson(context)=0
"""
test = db.gemd_db.execute_query(query)
db.print_and_dump(test, query, "invalid_jsons")

                                    uid  model_id        gemd_type   
0  b6d4ae78-3d79-4823-90cd-57c77be0ed19        11  measurement_run  \
1  24e460f0-54d1-4098-b1e4-93cb023704fc        11  measurement_run   

                                             context  
0  {"conditions": [], "file_links": [], "material...  
1  {"conditions": [], "file_links": [], "material...  


,uid,model_id,gemd_type,context
0,b6d4ae78-3d79-4823-90cd-57c77be0ed19,11,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."
1,24e460f0-54d1-4098-b1e4-93cb023704fc,11,measurement_run,"{""conditions"": [], ""file_links"": [], ""material..."


first advanced Birdshot query

In [39]:
query = f""" SELECT 
        i.uid, properties.name, properties.type, properties.nominal_value, properties.categorical_value, properties.units, properties.value, SplitJSON.*
        FROM gemdobject i 
        CROSS APPLY OPENJSON(JSON_QUERY(i.context, '$.properties')) 
        WITH 
        (
                name VARCHAR(50) '$.name', 
                value [nvarchar](max) '$.value' as json,
                nominal_value [nvarchar](4000) '$.value.nominal',
                categorical_value [nvarchar](4000) '$.value.category',
                units [nvarchar](4000) '$.value.units',
                type [nvarchar](32) '$.value.type'
        )
        AS properties 
        CROSS APPLY (
        SELECT
                SUBSTRING(value, 1, CHARINDEX('::', value) - 1) AS column_name,
                SUBSTRING(value, CHARINDEX('::', value) + 2, LEN(value)) AS column_value
        FROM OPENJSON(JSON_QUERY(i.context, '$.tags'))
        ) AS SplitJSON
        WHERE i.model_id={model_id} AND i.gemd_type='measurement_run' AND isjson(i.context)!=0"""

In [40]:
birdshot = db.gemd_db.execute_query(query)
db.print_and_dump(birdshot, query, "birdshot")

                                       uid                           name   
0     00b989a1-623e-4bd0-b673-462c6fe0a60b                   Weighed Mass  \
1     00b989a1-623e-4bd0-b673-462c6fe0a60b                   Weighed Mass   
2     00b989a1-623e-4bd0-b673-462c6fe0a60b                   Weighed Mass   
3     00b989a1-623e-4bd0-b673-462c6fe0a60b                      Mass Loss   
4     00b989a1-623e-4bd0-b673-462c6fe0a60b                      Mass Loss   
...                                    ...                            ...   
1705  fe25bc50-7563-468c-bbfc-3ef14442600d  Avg. Corrected Hardness (Gpa)   
1706  fe25bc50-7563-468c-bbfc-3ef14442600d  Avg. Corrected Hardness (Gpa)   
1707  fe25bc50-7563-468c-bbfc-3ef14442600d   Avg. Corrected Modulus (Gpa)   
1708  fe25bc50-7563-468c-bbfc-3ef14442600d   Avg. Corrected Modulus (Gpa)   
1709  fe25bc50-7563-468c-bbfc-3ef14442600d   Avg. Corrected Modulus (Gpa)   

              type     nominal_value categorical_value       units   
0    

,uid,name,type,nominal_value,categorical_value,units,value,column_name,column_value
0,00b989a1-623e-4bd0-b673-462c6fe0a60b,Weighed Mass,nominal_real,1.5588,None,gram,"{""nominal"": 1.5588, ""type"": ""nominal_real"", ""u...",composition_id,AAB08
1,00b989a1-623e-4bd0-b673-462c6fe0a60b,Weighed Mass,nominal_real,1.5588,None,gram,"{""nominal"": 1.5588, ""type"": ""nominal_real"", ""u...",batch,A
2,00b989a1-623e-4bd0-b673-462c6fe0a60b,Weighed Mass,nominal_real,1.5588,None,gram,"{""nominal"": 1.5588, ""type"": ""nominal_real"", ""u...",fabrication_method,VAM
3,00b989a1-623e-4bd0-b673-462c6fe0a60b,Mass Loss,nominal_real,0.0,None,gram,"{""nominal"": 0.0, ""type"": ""nominal_real"", ""unit...",composition_id,AAB08
4,00b989a1-623e-4bd0-b673-462c6fe0a60b,Mass Loss,nominal_real,0.0,None,gram,"{""nominal"": 0.0, ""type"": ""nominal_real"", ""unit...",batch,A
...,...,...,...,...,...,...,...,...,...
1705,fe25bc50-7563-468c-bbfc-3ef14442600d,Avg. Corrected Hardness (Gpa),nominal_real,2.28848475444444,None,gigapascal,"{""nominal"": 2.28848475444444, ""type"": ""nominal...",batch,A
1706,fe25bc50-7563-468c-bbfc-3ef14442600d,Avg. Corrected Hardness (Gpa),nominal_real,2.28848475444444,None,gigapascal,"{""nominal"": 2.28848475444444, ""type"": ""nominal...",fabrication_method,VAM
1707,fe25bc50-7563-468c-bbfc-3ef14442600d,Avg. Corrected Modulus (Gpa),nominal_real,169.115457311111,None,gigapascal,"{""nominal"": 169.115457311111, ""type"": ""nominal...",composition_id,AAB04
1708,fe25bc50-7563-468c-bbfc-3ef14442600d,Avg. Corrected Modulus (Gpa),nominal_real,169.115457311111,None,gigapascal,"{""nominal"": 169.115457311111, ""type"": ""nominal...",batch,A


query to curate measurements and measurement data, for optimal use by to birdshot

In [21]:
# query="""DECLARE @model_id INT; -- Define the model_id parameter
# SET @model_id = [YourModelIDValue]; -- Set the value of model_id

# SELECT
#     i.uid,
#     properties.name, 
#     properties.type,
#     properties.nominal_value, 
#     properties.categorical_value,
#     properties.units,
#     properties.value,
#     -- Add columns from the JSON tags array
#     MAX(CASE WHEN column_name = 'composition_id' THEN column_value END) AS composition_id,
#     MAX(CASE WHEN column_name = 'batch' THEN column_value END) AS batch,
#     MAX(CASE WHEN column_name = 'fabrication_method' THEN column_value END) AS fabrication_method,
#     -- Add columns from the JSON properties object
#     MAX(CASE WHEN property_name = 'name' THEN property_value END) AS property_name,
#     MAX(CASE WHEN property_name = 'json' THEN property_value END) AS property_json,
#     MAX(CASE WHEN property_name = 'nominal' THEN property_value END) AS property_nominal,
#     MAX(CASE WHEN property_name = 'category' THEN property_value END) AS property_category,
#     MAX(CASE WHEN property_name = 'units' THEN property_value END) AS property_units,
#     MAX(CASE WHEN property_name = 'type' THEN property_value END) AS property_type
# FROM GEMDOBject i
# -- Split the JSON tags array and add columns
# CROSS APPLY OPENJSON(JSON_QUERY(i.context, '$.tags'))
# WITH (
#     column_name NVARCHAR(50) '$' -- The tag contains both name and value
# ) AS tags_data
# CROSS APPLY (
#     SELECT 
#         SUBSTRING(tags_data.column_name, 1, CHARINDEX('::', tags_data.column_name) - 1) AS column_name,
#         SUBSTRING(tags_data.column_name, CHARINDEX('::', tags_data.column_name) + 2, LEN(tags_data.column_name)) AS column_value
# ) AS parsed_tags
# -- Extract properties from the JSON properties object
# CROSS APPLY OPENJSON(JSON_QUERY(i.context, '$.properties'))
# WITH (
#     property_name VARCHAR(50) '$.name', 
#     property_value [nvarchar](max) '$.value' as json,
#     property_nominal [nvarchar](4000) '$.value.nominal',
#     property_category [nvarchar](4000) '$.value.category',
#     property_units [nvarchar](4000) '$.value.units',
#     property_type [nvarchar](32) '$.value.type'
# ) AS properties
# WHERE i.model_id = @model_id
#     AND i.gemd_type = 'measurement_run'
#     AND ISJSON(i.context) = 1 -- Use '= 1' for ISJSON to check for valid JSON
# GROUP BY
#     i.uid,
#     properties.name, 
#     properties.type,
#     properties.nominal_value, 
#     properties.categorical_value,
#     properties.units,
#     properties.value; -- Group by the JSON property"""


In [22]:
# advanced_birdshot = db.gemd_db.execute_query(query)
# db.print_and_dump(advanced_birdshot, query, "advanced_birdshot")

TODO: further tailoring of query to specific measurements of interest for Bayesian Optimization